In [1]:
import pandas as pd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')
import time
from datetime import datetime, timedelta
from collections import defaultdict
from operator import itemgetter
import joblib
from joblib import dump, load
import json
import os
import pickle
import statistics as stats

import seaborn as sns
sns.set_style('darkgrid')
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
%matplotlib inline
import matplotlib.pyplot as plt
import pyodbc
import warnings
warnings.filterwarnings('ignore')

#data preprocessing and model:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
conn = cursor = None
try:
    conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};'
                'Server=eds-businessanalytics-sql01.database.windows.net;'
                'Database=AIP;'
                'UID=BAjones;'
                'PWD=5Laugh@funnymonkey!;')

    cursor = conn.cursor()  

    query = '''
            SELECT ci.*
               ,im.[System_Suspend_Flag] as 'system_suspend_flag'
               ,im.[Private_Brand_Flag] as 'private_brand_flag'
               ,im.[Dispenser_Flag] as 'dispenser_flag'
               ,im.[Proprietary_Flag] as 'proprietary_flag'
               ,im.[SuperFamily] as 'super_family'
               ,im.[FamilyGroup] as 'family_group'
               ,isnull(cri.[bin_revenue_product], 'NaN') as 'bin_rev_item'

            FROM [dbo].[CustomerItemStaging_202208] ci
            left join [dbo].[item_master_202208] im on im.[item_key] = ci.[item_key]
            left join [dbo].[classification_revenue_item_202208] cri on cri.[item_key] = ci.[Item_Key]
            '''
    cust_item_staging = pd.read_sql(query, conn)
finally:
    try: cursor.close()
    except: pass
    try: conn.close()
    except: pass

In [3]:
## Customer Master Table AIP

conn = cursor = None
try:
    conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};'
                'Server=eds-businessanalytics-sql01.database.windows.net;'
                'Database=AIP;'
                'UID=BAjones;'
                'PWD=5Laugh@funnymonkey!;')

    cursor = conn.cursor()  
    
    query = '''
            SELECT cm.[Customer_Key] as 'customer_key'
                  ,cm.[Region] as 'region'
                  ,cm.[Primary_Sales_Rep_Key] as 'sales_rep_key'
                  ,isnull(cm.[price_type], 'M') as 'price_type'
                  ,isnull(cm.[suspend_flag],'N') as 'suspend_flag'
                  ,isnull(cm.[distributor_negotiated_cost_flag], 'Y') as 'distributor_negotiated_cost_flag'
                  ,cm.[Integrated_Market_Vertical] as 'int_market_vert'
                  ,isnull(crc.[bin_revenue_customer], 'NaN') as 'bin_rev_cust'
            FROM [dbo].[customer_master_202208] cm
            left join [archive].[classification_revenue_customer_202208] crc on crc.[customer_key] = cm.[customer_key]
            '''
    cust_mast = pd.read_sql(query, conn)
finally:
    try: cursor.close()
    except: pass
    try: conn.close()
    except: pass

In [4]:
result = pd.read_csv('prioritized_data_202208.csv')

In [5]:
staging = pd.merge(cust_item_staging, cust_mast, on=['customer_key'],how='left')

In [6]:
staging['CustItem'] = staging['customer_key'].astype(str) + staging['item_key'].astype(str)

In [7]:
result['CustItem'] = result['CustItem'].astype(str)

In [8]:
def preprocess_data(df):

    df['customer_key'] = df['customer_key'].astype(str)
    df['item_key'] = df['item_key'].astype(str)
    # df['item_key'] = df['item_key'].str[:-2]

    df['sales_rep_key'] = df['sales_rep_key'].astype(str)
    #df['sales_rep_key'] = df['sales_rep_key'].str[:-2]

    df['identity_key'] = df['customer_key'].astype(str) + '/' + df['item_key'].astype(str) + '/' + df['sales_rep_key'].astype(str)
    df['cust_item_key'] = df['customer_key'].astype(str) + '/' + df['item_key'].astype(str)

    df['price_type'] = df['price_type'].replace(np.nan, 'M')
    df['suspend_flag'] = df['suspend_flag'].replace(np.nan, 'N')
    df['system_suspend_Flag'] = df['system_suspend_flag'].replace(np.nan, 'N')

    df = df[df['price_type']=='M']
    df = df[df['suspend_flag']=='N']
    df = df[df['system_suspend_flag']=='N']
    df = df[df['DateOfLastPresentation'].isnull()]

    
    df = df[df['stretch_margin_percentage'] < 0.85] 

    df['LastPriceChange'] = df['LastPriceChange'].astype(str)
    df['LastPriceChange'] = df['LastPriceChange'].str[0:8]
    df['LastPriceChange'] = pd.to_datetime(df['LastPriceChange'])
    df['DateOfLastPresentation'] = pd.to_datetime(df['DateOfLastPresentation'])
 
    now = datetime.now() # current date and time
    df['TimeDelta'] = now.strftime("%Y-%m-%d")
    df['TimeDelta'] = pd.to_datetime(df['TimeDelta'])


    df['LastPriceChange_delta'] = df['TimeDelta'] - df['LastPriceChange']
    df['DateOfLastPresentation_delta'] = df['TimeDelta'] - df['DateOfLastPresentation']
  
    df['DateOfLastPresentation_delta'] = df['DateOfLastPresentation_delta'].fillna(pd.Timedelta('365 days')) #change to 365
    df['LastPriceChange_delta'] = df['LastPriceChange_delta'].fillna(pd.Timedelta('365 days'))

    df['DateOfLastPresentation_delta'] = df['DateOfLastPresentation_delta'].dt.days.astype('int16')
    df['LastPriceChange_delta'] = df['LastPriceChange_delta'].dt.days.astype('int16')

    df['price_change'] = np.where(df['LastPriceChange_delta']==365, 'yes', 'no')
    #df = df[df['DateOfLastPresentation_delta'] > 120] 


    conditions_for_price_change = [
     (df['LastPriceChange_delta'] > 0) & (df['LastPriceChange_delta'] <= 30),
     (df['LastPriceChange_delta'] > 30) & (df['LastPriceChange_delta'] <=60),
     (df['LastPriceChange_delta'] > 60) & (df['LastPriceChange_delta'] <=90),
     (df['LastPriceChange_delta'] > 90) & (df['LastPriceChange_delta'] <=120),
     (df['LastPriceChange_delta'] > 150) & (df['LastPriceChange_delta'] <=180),
     (df['LastPriceChange_delta'] > 210) & (df['LastPriceChange_delta'] <=240),
     (df['LastPriceChange_delta'] > 240) & (df['LastPriceChange_delta'] <=270),
     (df['LastPriceChange_delta'] > 270) & (df['LastPriceChange_delta'] <=300),
     (df['LastPriceChange_delta'] > 300) & (df['LastPriceChange_delta'] <=330),
     (df['LastPriceChange_delta'] > 330) & (df['LastPriceChange_delta'] <=360),
     (df['LastPriceChange_delta'] > 360) 
    ]
    values_for_price_change = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

    conditions_for_last_presentation = [
     (df['DateOfLastPresentation_delta'] <= 0),
     (df['DateOfLastPresentation_delta'] > 0) & (df['DateOfLastPresentation_delta'] <= 30),
     (df['DateOfLastPresentation_delta'] > 30) & (df['DateOfLastPresentation_delta'] <=60),
     (df['DateOfLastPresentation_delta'] > 60) & (df['DateOfLastPresentation_delta'] <=90),
     (df['DateOfLastPresentation_delta'] > 90) & (df['DateOfLastPresentation_delta'] <=120),
     (df['DateOfLastPresentation_delta'] > 150) & (df['DateOfLastPresentation_delta'] <=180),
     (df['DateOfLastPresentation_delta'] > 210) & (df['DateOfLastPresentation_delta'] <=240),
     (df['DateOfLastPresentation_delta'] > 240) & (df['DateOfLastPresentation_delta'] <=270),
     (df['DateOfLastPresentation_delta'] > 270) & (df['DateOfLastPresentation_delta'] <=300),
     (df['DateOfLastPresentation_delta'] > 300) & (df['DateOfLastPresentation_delta'] <=330),
     (df['DateOfLastPresentation_delta'] > 330) & (df['DateOfLastPresentation_delta'] <=360),
     (df['DateOfLastPresentation_delta'] > 360) 
    ]   
    values_for_last_presentation = [50, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


    df['LastPriceChange_points'] = np.select(conditions_for_price_change, values_for_price_change)
    df['DateOfLastPresentation_points'] = np.select(conditions_for_last_presentation, values_for_last_presentation)

    df['recency_points'] = df['DateOfLastPresentation_points'] + df['LastPriceChange_points'] 


    df['floor_price'] = df['CurrentCost'] / (1 - df['floor_margin_percentage'])
    df['target_price'] = df['CurrentCost'] / (1 - df['target_margin_percentage'])
    df['stretch_price'] = df['CurrentCost'] / (1 - df['stretch_margin_percentage'])

    df['floor_diff'] = df['floor_price'] - df['CurrentPrice']  
    df['target_diff'] = df['target_price'] - df['CurrentPrice'] 
    df['stretch_diff'] = df['stretch_price']- df['CurrentPrice'] 

    diff_cols = ['floor_diff','target_diff','stretch_diff']
    df[diff_cols] = df[diff_cols].replace(np.nan,0)

    df['opportunity_diff'] = df['floor_diff'] + df['target_diff'] + df['stretch_diff']


    df = df.drop_duplicates()

    return df

In [9]:
staging1 = preprocess_data(staging)

In [10]:
staging1.shape

(602408, 44)

In [11]:
result.shape

(515950, 70)

In [12]:
def data_engineering(df):
    df = df[['identity_key','_12mo_Rev','lift_overall','floor_margin_percentage','target_margin_percentage','stretch_margin_percentage',
             'private_brand_flag','dispenser_flag','proprietary_flag','super_family','family_group','bin_rev_item','region',
             'distributor_negotiated_cost_flag','int_market_vert','bin_rev_cust','price_change']]
    scaler = StandardScaler()
    num_cols = ['_12mo_Rev','lift_overall','floor_margin_percentage','target_margin_percentage','stretch_margin_percentage']
    df[num_cols] = scaler.fit_transform(df[num_cols])
    encoder = OneHotEncoder(sparse=False)
    cat_cols = ['private_brand_flag','dispenser_flag','proprietary_flag','super_family','family_group','bin_rev_item','region',
                'distributor_negotiated_cost_flag','int_market_vert','bin_rev_cust','price_change']
    df[cat_cols] = df[cat_cols].replace(np.nan, 'Unknown')

    df_encoded = pd.DataFrame(encoder.fit_transform(df[cat_cols]))
    df_encoded.columns = encoder.get_feature_names(cat_cols)

    df = df.drop(cat_cols ,axis=1)
    df = pd.concat([df_encoded, df], axis=1)

    df.dropna(inplace=True)

    return df 

In [13]:
data_engineered = data_engineering(staging1)

In [14]:
def rf_model_predict(df):
    path = '../PD Function/classification/'
    model = load(path+'Price_Optimizer_RandomForestGrid_Classification.joblib')   
    #X_train = pd.read_csv(path+'x_train.csv')   
    identity_key = df['identity_key']
    df = df.drop(columns = ['identity_key'])

    predictions = pd.DataFrame()
    y_pred = model.predict(df)
    predictions = pd.concat([identity_key, df], axis = 1).dropna()
    predictions['prediction'] = y_pred
    return predictions

In [15]:
data_preds = rf_model_predict(data_engineered)

In [16]:
preds_split = data_preds['identity_key'].str.split('/', n=2, expand=True)
data_preds['customer_key'] = preds_split[0]
data_preds['item_key'] = preds_split[1]
data_preds['sales_rep_key'] = preds_split[2]
preds = data_preds[['identity_key','prediction']]
staging2 = pd.merge(staging1, preds, on=['identity_key'],how='left')  

staging2['predictions_points'] = np.where(staging2['prediction']== 1, 100, 0)

In [17]:
staging2['12mo_rev_rank'] =staging2['_12mo_Rev'].rank(method='min')
staging2['3mo_proj_demand_rank'] = staging2['_3mo_Projected_Demand'].rank(method='min')
staging2['opportunity_rank'] = staging2['opportunity_diff'].rank(method='min')
staging2['lift_overall_rank'] = staging2['lift_overall'].rank(method='min')

In [18]:
staging2.shape

(602408, 50)

In [19]:
sales = pd.read_csv('sales_clean_06.csv')

In [20]:
sales['Date'] = pd.to_datetime(sales['Date'])

In [21]:
sales['CustItem'] = sales['customer_key'].astype(str) + sales['item_key'].astype(str)
sales = sales[['CustItem','Shipped Quantity','Sales Amount','Date']]

In [22]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [27]:
cust_item = pd.DataFrame(staging2['CustItem'].unique())
cust_item.columns = ['CustItem']

In [28]:
cust_item.shape

(602408, 1)

In [34]:
cust_item = pd.DataFrame(sales['CustItem'].unique())
cust_item.columns = ['CustItem']

In [35]:
cust_item2.shape

(646826, 1)

In [36]:
max_purchase = sales.groupby('CustItem').Date.max().reset_index()
max_purchase.columns = ['CustItem','MaxPurchaseDate']
max_purchase['Recency'] = (max_purchase['MaxPurchaseDate'].max() - max_purchase['MaxPurchaseDate']).dt.days
cust_item = pd.merge(cust_item, max_purchase[['CustItem','Recency']], on='CustItem')
kmeans = KMeans(n_clusters=4)
kmeans.fit(cust_item[['Recency']])
cust_item['RecencyCluster'] = kmeans.predict(cust_item[['Recency']])
cust_item = order_cluster('RecencyCluster', 'Recency',cust_item,False)

In [37]:
cust_item.shape

(646826, 3)

In [38]:
frequency = sales.groupby('CustItem').Date.count().reset_index()
frequency.columns = ['CustItem','Frequency']
cust_item = pd.merge(cust_item, frequency, on='CustItem')
kmeans = KMeans(n_clusters=5)
kmeans.fit(cust_item[['Frequency']])
cust_item['FrequencyCluster'] = kmeans.predict(cust_item[['Frequency']])
cust_item = order_cluster('FrequencyCluster', 'Recency',cust_item,False)

In [39]:
cust_item.shape

(646826, 5)

In [40]:
sales['Revenue'] = sales['Sales Amount']
revenue = sales.groupby('CustItem')['Revenue'].sum().reset_index()
cust_item = pd.merge(cust_item, revenue, on='CustItem')
#apply clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(cust_item[['Revenue']])
cust_item['RevenueCluster'] = kmeans.predict(cust_item[['Revenue']])
cust_item = order_cluster('RevenueCluster', 'Revenue',cust_item,True)

In [41]:
cust_item.shape

(646826, 7)

In [42]:
sales['Density'] = sales['Shipped Quantity']
density = sales.groupby('CustItem')['Density'].sum().reset_index()
cust_item = pd.merge(cust_item, density, on='CustItem')
kmeans = KMeans(n_clusters=5)
kmeans.fit(cust_item[['Density']])
cust_item['DensityCluster'] = kmeans.predict(cust_item[['Density']])
cust_item = order_cluster('DensityCluster', 'Density',cust_item,True)

In [43]:
cust_item.shape

(646826, 9)

In [44]:
_12mo_rev_rank = staging2[['12mo_rev_rank','CustItem']]
cust_item = pd.merge(cust_item, _12mo_rev_rank,on='CustItem',)
kmeans = KMeans(n_clusters=5)
kmeans.fit(cust_item[['12mo_rev_rank']])
cust_item['12MonthCluster'] = kmeans.predict(cust_item[['12mo_rev_rank']])
cust_item = order_cluster('12MonthCluster', '12mo_rev_rank',cust_item,True)

In [56]:
_12mo_rev_rank.shape

(602408, 2)

In [57]:
cust_item.shape

(515950, 11)

In [58]:
_3mo_usage_rank = staging2[['3mo_proj_demand_rank','CustItem']]
cust_item = pd.merge(cust_item, _3mo_usage_rank,on='CustItem')
kmeans = KMeans(n_clusters=3)
kmeans.fit(cust_item[['3mo_proj_demand_rank']])
cust_item['3MonthUsageCluster'] = kmeans.predict(cust_item[['3mo_proj_demand_rank']])
cust_item = order_cluster('3MonthUsageCluster', '3mo_proj_demand_rank',cust_item,True)

In [59]:
cust_item.shape

(515950, 13)

In [60]:
opportunity_rank = staging2[['opportunity_rank','CustItem']]
cust_item = pd.merge(cust_item, opportunity_rank,on='CustItem')
kmeans = KMeans(n_clusters=4)
kmeans.fit(cust_item[['opportunity_rank']])
cust_item['OpportunityCluster'] = kmeans.predict(cust_item[['opportunity_rank']])
cust_item = order_cluster('OpportunityCluster', 'opportunity_rank',cust_item,True)

In [61]:
cust_item.shape

(515950, 15)

In [62]:
lift_overall_rank = staging2[['lift_overall_rank','CustItem']]
cust_item = pd.merge(cust_item, lift_overall_rank,on='CustItem')
kmeans = KMeans(n_clusters=5)
kmeans.fit(cust_item[['lift_overall_rank']])
cust_item['LiftCluster'] = kmeans.predict(cust_item[['lift_overall_rank']])
cust_item = order_cluster('LiftCluster', 'lift_overall_rank',cust_item,True)

In [63]:
cust_item.shape

(515950, 17)

In [64]:
cust_item.head()

,CustItem,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,Density,DensityCluster,12mo_rev_rank,12MonthCluster,3mo_proj_demand_rank,3MonthUsageCluster,opportunity_rank,OpportunityCluster,lift_overall_rank,LiftCluster
0,407185214973,149,2,1,0,40.92,0,1.0,0,112620.0,0,155.0,0,569405.0,3,147924.0,1
1,407779278088,189,2,1,0,39.56,0,4.0,0,108505.0,0,155.0,0,464574.0,3,211904.0,1
2,407780218317,121,2,2,0,71.45,0,2.0,0,100776.0,0,155.0,0,494576.0,3,162152.0,1
3,408360223893,148,2,2,0,57.46,0,2.0,0,76171.0,0,155.0,0,584628.0,3,191327.0,1
4,4084283335823,199,2,1,0,16.25,0,1.0,0,39331.0,0,155.0,0,559944.0,3,182556.0,1


In [65]:
staging3 = pd.merge(cust_item, staging2, on=['CustItem'],how='left')

In [66]:
staging3['OverallScore'] = staging3['RecencyCluster'] + staging3['FrequencyCluster'] + staging3['RevenueCluster'] + staging3['DensityCluster'] + staging3['12MonthCluster'] + staging3['3MonthUsageCluster'] + staging3['LiftCluster'] + staging3['OpportunityCluster']
staging3.rename(columns={'OverallCluster':'OverallScore'},inplace=True)
staging3['OverallClusterPoints'] = staging3['OverallScore'] * 100
staging3['PriorityPoints'] = staging3['recency_points'] + staging3['OverallClusterPoints'] + staging3['predictions_points']
staging3['priority_determination'] = staging3['PriorityPoints']
staging3['priority_determination'] = (staging3['priority_determination']-staging3['priority_determination'].min())/(staging3['priority_determination'].max()-staging3['priority_determination'].min())

In [67]:
staging3.shape

(515950, 70)

In [68]:
result.shape

(515950, 70)

In [69]:
staging.shape

(684557, 30)

In [70]:
def preprocess_data_all(df):

    df['customer_key'] = df['customer_key'].astype(str)
    df['item_key'] = df['item_key'].astype(str)
    # df['item_key'] = df['item_key'].str[:-2]

    df['sales_rep_key'] = df['sales_rep_key'].astype(str)
    #df['sales_rep_key'] = df['sales_rep_key'].str[:-2]

    df['identity_key'] = df['customer_key'].astype(str) + '/' + df['item_key'].astype(str) + '/' + df['sales_rep_key'].astype(str)
    df['cust_item_key'] = df['customer_key'].astype(str) + '/' + df['item_key'].astype(str)

    df['price_type'] = df['price_type'].replace(np.nan, 'M')
    df['suspend_flag'] = df['suspend_flag'].replace(np.nan, 'N')
    df['system_suspend_Flag'] = df['system_suspend_flag'].replace(np.nan, 'N')

    df['item_suspended'] = np.where(df['system_suspend_flag']=='N', 'PASS', 'FAIL')
    df['cust_suspended'] = np.where(df['suspend_flag']=='N', 'PASS', 'FAIL')
    df['presented'] = np.where(df['DateOfLastPresentation'].isnull(), 'PASS', 'FAIL')
    df['margin_cut_off'] = np.where(df['stretch_margin_percentage'] < 0.85, 'PASS', 'FAIL')


    df['floor_price'] = df['CurrentCost'] / (1 - df['floor_margin_percentage'])
    df['target_price'] = df['CurrentCost'] / (1 - df['target_margin_percentage'])
    df['stretch_price'] = df['CurrentCost'] / (1 - df['stretch_margin_percentage'])

    df['floor_diff'] = df['floor_price'] - df['CurrentPrice']  
    df['target_diff'] = df['target_price'] - df['CurrentPrice'] 
    df['stretch_diff'] = df['stretch_price']- df['CurrentPrice'] 

    diff_cols = ['floor_diff','target_diff','stretch_diff']
    df[diff_cols] = df[diff_cols].replace(np.nan,0)

    df['opportunity_diff'] = df['floor_diff'] + df['target_diff'] + df['stretch_diff']

    return df

In [71]:
test = preprocess_data_all(staging)

In [72]:
test.shape

(684557, 41)

In [73]:
staging.shape

(684557, 41)

In [74]:
test['price_type'].value_counts()

M    684557
Name: price_type, dtype: int64

In [75]:
test['suspend_flag'].value_counts()


N    681621
Y      2936
Name: suspend_flag, dtype: int64

In [76]:
test['system_suspend_flag'].value_counts()


N    682884
Y      1673
Name: system_suspend_flag, dtype: int64

In [77]:
presented = test[~test['DateOfLastPresentation'].isnull()]

In [78]:
presented.shape

(64207, 41)

In [79]:
margin_cut_off = staging[staging['stretch_margin_percentage'] > 0.85] 

In [80]:
margin_cut_off.shape

(1359, 41)

In [81]:
test.head()

,customer_key,item_key,_12mo_Rev,_3mo_Projected_Demand,CurrentPrice,CurrentCost,LastPriceChange,floor_margin_percentage,target_margin_percentage,stretch_margin_percentage,...,cust_suspended,presented,margin_cut_off,floor_price,target_price,stretch_price,floor_diff,target_diff,stretch_diff,opportunity_diff
0,451150,228618,424.00,0.0,212.00,95.4000,NaN,0.22729,0.30281,0.38911,...,PASS,PASS,PASS,123.461583,136.835009,156.165594,-88.538417,-75.164991,-55.834406,-219.537813
1,467599,228618,292.54,0.0,146.27,95.4000,NaN,0.17752,0.27659,0.48464,...,PASS,PASS,PASS,115.990662,131.875423,185.113319,-30.279338,-14.394577,38.843319,-5.830595
2,417538,228618,858.46,0.0,95.62,86.0600,20220407.0,0.23041,0.33708,0.41836,...,PASS,PASS,PASS,111.825777,129.819586,147.960938,16.205777,34.199586,52.340938,102.746301
3,459406,228620,440.95,2.0,88.19,81.3433,NaN,0.21882,0.34576,0.45317,...,PASS,PASS,PASS,104.128754,124.332508,148.754275,15.938754,36.142508,60.564275,112.645537
4,451362,228620,736.62,0.0,245.54,147.6600,NaN,0.21882,0.34576,0.45317,...,PASS,PASS,PASS,189.021736,225.696992,270.029077,-56.518264,-19.843008,24.489077,-51.872195


In [82]:
cust_item.head()

,CustItem,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,Density,DensityCluster,12mo_rev_rank,12MonthCluster,3mo_proj_demand_rank,3MonthUsageCluster,opportunity_rank,OpportunityCluster,lift_overall_rank,LiftCluster
0,407185214973,149,2,1,0,40.92,0,1.0,0,112620.0,0,155.0,0,569405.0,3,147924.0,1
1,407779278088,189,2,1,0,39.56,0,4.0,0,108505.0,0,155.0,0,464574.0,3,211904.0,1
2,407780218317,121,2,2,0,71.45,0,2.0,0,100776.0,0,155.0,0,494576.0,3,162152.0,1
3,408360223893,148,2,2,0,57.46,0,2.0,0,76171.0,0,155.0,0,584628.0,3,191327.0,1
4,4084283335823,199,2,1,0,16.25,0,1.0,0,39331.0,0,155.0,0,559944.0,3,182556.0,1


In [83]:
test.columns

Index(['customer_key', 'item_key', '_12mo_Rev', '_3mo_Projected_Demand',
       'CurrentPrice', 'CurrentCost', 'LastPriceChange',
       'floor_margin_percentage', 'target_margin_percentage',
       'stretch_margin_percentage', 'lift_overall', 'DateOfLastPresentation',
       'system_suspend_flag', 'private_brand_flag', 'dispenser_flag',
       'proprietary_flag', 'super_family', 'family_group', 'bin_rev_item',
       'region', 'sales_rep_key', 'price_type', 'suspend_flag',
       'distributor_negotiated_cost_flag', 'int_market_vert', 'bin_rev_cust',
       'CustItem', 'identity_key', 'cust_item_key', 'system_suspend_Flag',
       'item_suspended', 'cust_suspended', 'presented', 'margin_cut_off',
       'floor_price', 'target_price', 'stretch_price', 'floor_diff',
       'target_diff', 'stretch_diff', 'opportunity_diff'],
      dtype='object')

In [85]:
test2 = test[['CustItem','customer_key', 'item_key', '_12mo_Rev', '_3mo_Projected_Demand','CurrentPrice', 'CurrentCost', 'LastPriceChange','lift_overall', 'DateOfLastPresentation',
       'system_suspend_flag', 'bin_rev_item','region', 'sales_rep_key', 'price_type', 'suspend_flag','bin_rev_cust', 'item_suspended', 'cust_suspended', 'presented', 'margin_cut_off',
       'floor_price','target_price', 'stretch_price']]

In [86]:
test_dups = test2[test2.duplicated()]

In [87]:
test_dups.shape

(12910, 24)

In [88]:
test_dups['duplication'] = 'FAIL'

In [89]:
test_unique = test2.drop_duplicates()

In [90]:
test_unique['duplication'] = 'PASS'

In [91]:
test3 = pd.concat([test_unique, test_dups])

In [92]:
test3.head()

,CustItem,customer_key,item_key,_12mo_Rev,_3mo_Projected_Demand,CurrentPrice,CurrentCost,LastPriceChange,lift_overall,DateOfLastPresentation,...,suspend_flag,bin_rev_cust,item_suspended,cust_suspended,presented,margin_cut_off,floor_price,target_price,stretch_price,duplication
0,451150228618,451150,228618,424.00,0.0,212.00,95.4000,NaN,1679.958,NaT,...,N,C,PASS,PASS,PASS,PASS,123.461583,136.835009,156.165594,PASS
1,467599228618,467599,228618,292.54,0.0,146.27,95.4000,NaN,4344.335,NaT,...,N,B,PASS,PASS,PASS,PASS,115.990662,131.875423,185.113319,PASS
2,417538228618,417538,228618,858.46,0.0,95.62,86.0600,20220407.0,754.487,NaT,...,N,B,PASS,PASS,PASS,PASS,111.825777,129.819586,147.960938,PASS
3,459406228620,459406,228620,440.95,2.0,88.19,81.3433,NaN,1884.828,NaT,...,N,C,PASS,PASS,PASS,PASS,104.128754,124.332508,148.754275,PASS
4,451362228620,451362,228620,736.62,0.0,245.54,147.6600,NaN,1884.828,NaT,...,N,C,PASS,PASS,PASS,PASS,189.021736,225.696992,270.029077,PASS


In [95]:
test4 = pd.merge(test3, cust_item, on=['CustItem'], how='left')

In [150]:
test4.to_csv('test4.csv',index=False)

In [98]:
test5 = test4.drop(columns=['DateOfLastPresentation', '12mo_rev_rank', '3mo_proj_demand_rank','opportunity_rank','lift_overall_rank'])

In [99]:
test5.drop(columns=['CurrentPrice', 'CurrentCost', 'system_suspend_flag','price_type', 'suspend_flag','floor_price', 'target_price','stretch_price'],inplace=True)

In [100]:
test5.head(10)

,CustItem,customer_key,item_key,_12mo_Rev,_3mo_Projected_Demand,LastPriceChange,lift_overall,bin_rev_item,region,sales_rep_key,...,Frequency,FrequencyCluster,Revenue,RevenueCluster,Density,DensityCluster,12MonthCluster,3MonthUsageCluster,OpportunityCluster,LiftCluster
0,451150228618,451150,228618,424.00,0.0,NaN,1679.958,D,West,2518.0,...,4.0,0.0,424.00,0.0,2.0,0.0,3.0,0.0,0.0,2.0
1,467599228618,467599,228618,292.54,0.0,NaN,4344.335,D,West,1982.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,417538228618,417538,228618,858.46,0.0,20220407.0,754.487,D,West,2288.0,...,15.0,1.0,1999.54,0.0,21.0,0.0,4.0,0.0,3.0,1.0
3,459406228620,459406,228620,440.95,2.0,NaN,1884.828,D,West,1840.0,...,7.0,0.0,529.14,0.0,6.0,0.0,3.0,1.0,3.0,2.0
4,451362228620,451362,228620,736.62,0.0,NaN,1884.828,D,West,2023.0,...,2.0,0.0,736.62,0.0,3.0,0.0,4.0,0.0,0.0,2.0
5,474248228620,474248,228620,143.78,0.0,NaN,1302.263,D,West,1841.0,...,1.0,0.0,143.78,0.0,1.0,0.0,2.0,0.0,0.0,2.0
6,448000228620,448000,228620,3159.92,0.0,20220224.0,1884.828,D,West,1849.0,...,23.0,1.0,4603.70,0.0,41.0,0.0,4.0,0.0,0.0,2.0
7,442274228620,442274,228620,173.72,0.0,NaN,592.747,D,West,2220.0,...,2.0,0.0,173.72,0.0,1.0,0.0,2.0,0.0,3.0,1.0
8,434042228620,434042,228620,111.86,0.0,NaN,705.362,D,West,2394.0,...,1.0,0.0,111.86,0.0,1.0,0.0,2.0,0.0,3.0,1.0
9,418327228620,418327,228620,625.70,0.0,NaN,1897.476,D,West,2533.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
sales.head()

,CustItem,Shipped Quantity,Sales Amount,Date,Revenue,Density
0,407185214973,1.0,40.92,2022-03-02,40.92,1.0
1,407185217763,1.0,31.56,2021-06-21,31.56,1.0
2,407185220008,1.0,45.97,2021-06-21,45.97,1.0
3,407185220031,1.0,41.39,2021-10-12,41.39,1.0
4,407185220187,1.0,78.64,2020-11-16,78.64,1.0


In [102]:
fact_sales = sales.groupby('CustItem')['Shipped Quantity','Sales Amount'].sum().reset_index()

In [103]:
fact_sales.head()

,CustItem,Shipped Quantity,Sales Amount
0,19930603258059,200.0,416.00
1,19930603258560,7.0,90.30
2,19930603259381,25.0,491.50
3,19930603259517,7.0,270.55
4,19930603259548,3.0,415.17


In [104]:
fact_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646826 entries, 0 to 646825
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CustItem          646826 non-null  object 
 1   Shipped Quantity  646826 non-null  float64
 2   Sales Amount      646826 non-null  float64
dtypes: float64(2), object(1)
memory usage: 14.8+ MB


In [105]:
fact_sales['Shipped Quantity'].min()

-3229.0

In [106]:
test6 = pd.merge(test5, fact_sales, on=['CustItem'],how='left')

In [173]:
#test6.to_csv('test6.csv',index=False)

In [107]:
test6["Recency"].fillna(-99999, inplace = True)
test6["RecencyCluster"].fillna(-99999, inplace = True)
test6["Frequency"].fillna(-99999, inplace = True)
test6["FrequencyCluster"].fillna(-99999, inplace = True)
test6["Revenue"].fillna(-99999, inplace = True)
test6["RevenueCluster"].fillna(-99999, inplace = True)
test6["Density"].fillna(-99999, inplace = True)
test6["DensityCluster"].fillna(-99999, inplace = True)
test6["12MonthCluster"].fillna(-99999, inplace = True)
test6["3MonthUsageCluster"].fillna(-99999, inplace = True)
test6["OpportunityCluster"].fillna(-99999, inplace = True)
test6["LiftCluster"].fillna(-99999, inplace = True)
test6["Shipped Quantity"].fillna(-99999, inplace = True)
test6["Sales Amount"].fillna(-99999, inplace = True)

In [109]:
stage1_pass = test6[test6['duplication']=='PASS']
stage1_fail = test6[test6['duplication']=='FAIL']

In [110]:
stage2_pass = stage1_pass[stage1_pass['item_suspended'] == 'PASS']
stage2_fail = stage1_pass[stage1_pass['item_suspended'] == 'FAIL']

In [111]:
stage3_pass = stage2_pass[stage2_pass['cust_suspended'] == 'PASS']
stage3_fail = stage2_pass[stage2_pass['cust_suspended'] == 'FAIL']

In [112]:
stage4_pass = stage3_pass[stage3_pass['presented'] == 'PASS']
stage4_fail = stage3_pass[stage3_pass['presented'] == 'FAIL']

In [113]:
stage5_pass = stage4_pass[stage4_pass['Sales Amount'] > -99999]
stage5_fail = stage4_pass[stage4_pass['Sales Amount'] == -99999]

In [114]:
stage6_pass = stage5_pass[stage5_pass['Revenue'] > -99999]
stage6_fail = stage5_pass[stage5_pass['Revenue'] == -99999]

In [115]:
stage6_fail_list = list(stage6_fail['CustItem'])

In [116]:
checking = fact_sales.loc[fact_sales['CustItem'].isin(stage6_fail_list)]

In [117]:
checking.shape

(1227, 3)

In [120]:
stage6_fail = stage6_fail.sort_values(by='CustItem',ascending=True)

In [ ]:
#count of combination and dollar value 

In [125]:
starting_data_shape = 684557
ending_data_shape = 515950

# PASS 
stage1_pass_shape = 671647
stage2_pass_shape = 669980
stage3_pass_shape = 667056
stage4_pass_shape = 603750
stage5_pass_shape = 517177
stage6_pass_shape = 515950

# FAIL
stage1_fail_shape = 12910
stage2_fail_shape = 1667
stage3_fail_shape = 2924
stage4_fail_shape = 63306
stage5_fail_shape = 86573
stage6_fail_shape = 1227

In [230]:
print(f'Starting Data: {staging_shape}.')
print(f'Dropped Duplicates: {test_no_dups_shape}.')

Starting Data: 684557.
Dropped Duplicates: 671647.


In [238]:
print(f'Starting Data: {staging_shape}.')
print(f'Number of Duplicates: {test_dups_shape}.')

Starting Data: 684557.
Number of Duplicates: 12910.
